Let's collab here on building the predictor.

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
import pandas as pd
import numpy as np
#data=pd.read_csv("/content/drive/Shareddrives/EMSE 6574 Python Project/spotify.csv")
# data=pd.read_csv('/content/drive/MyDrive/Copy of final.csv')
data = pd.read_csv('/Users/andrewpaladino/Downloads/final_spotify_data.csv')

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

from sklearn.preprocessing import OneHotEncoder

In [36]:
data

,Unnamed: 0,genre,artist_name,track_name,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,duration_s,Unnamed: 0.1
0,1159,Alternative,Nickelback,How You Remind Me,75,0.00135,0.446,0.764,0.000000,0.0990,-5.042,0.0330,172.094,0.543,3730.666667,51.0
1,7012,Country,Andy Gibb,Shadow Dancing,46,0.14200,0.742,0.600,0.000000,0.0677,-9.647,0.0319,101.920,0.954,4600.000000,55.0
2,13501,Dance,Mariah Carey,We Belong Together,72,0.03580,0.838,0.469,0.000000,0.0928,-7.992,0.0835,139.975,0.778,3356.666667,15.0
3,13522,Dance,Mario,Let Me Love You,80,0.23500,0.656,0.578,0.000000,0.1180,-8.970,0.0922,94.514,0.556,4278.883333,65.0
4,13570,Dance,The Black Eyed Peas,I Gotta Feeling,70,0.08230,0.741,0.748,0.000000,0.3400,-6.299,0.0264,127.965,0.600,4818.883333,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176509,176509,Soul,H-Town,They Like It Slow,39,0.11300,0.611,0.493,0.000000,0.1050,-9.297,0.0354,115.920,0.443,4658.500000,NaN
176510,176510,Soul,John Legend,Quickly (feat. Brandy),39,0.23100,0.736,0.701,0.000000,0.2030,-4.345,0.1000,99.991,0.770,3711.116667,NaN
176511,176511,Soul,Belly,P.O.P.,43,0.10400,0.802,0.516,0.000485,0.1050,-9.014,0.2130,175.666,0.482,3352.883333,NaN
176512,176512,Soul,Jr Thomas & The Volcanos,Burning Fire,38,0.03290,0.785,0.683,0.000880,0.2370,-6.944,0.0337,113.830,0.969,4707.450000,NaN


In [6]:
data.genre.unique()

array(['Alternative', 'Country', 'Dance', 'Folk', 'Blues', 'Hip-Hop',
       'Pop', 'Rock', 'Soul', 'Movie', 'R&B', 'A Capella', 'Electronic',
       'Anime', 'Opera', "Children's Music", 'Children’s Music', 'Rap',
       'Indie', 'Classical', 'Reggae', 'Reggaeton', 'Jazz', 'Ska',
       'Comedy', 'Soundtrack', 'World'], dtype=object)

In [8]:
movie_song = data[['genre', 'track_name']]

In [31]:
movie_song[(movie_song.genre == 'Soundtrack')].sample(10)

,genre,track_name
153989,Soundtrack,"Pay The Iron Price - From The ""Game Of Thrones..."
153200,Soundtrack,Victory Does Not Make Us Conquerors - From The...
153121,Soundtrack,Procession
155697,Soundtrack,The Primer
158206,Soundtrack,Trooping With Crows
159254,Soundtrack,New Message
154755,Soundtrack,Noah Visits - Score
152436,Soundtrack,Main Title (The Godfather Waltz)
159882,Soundtrack,Y.A.M. -Young AllMight-
158866,Soundtrack,"Alice Reprise #5 - From ""Alice in Wonderland""/..."


In [40]:
data_two = data[(data.genre != "Children's Music") & (data.genre != "Comedy") & (data.genre != "World") & (data.genre != "'Children's Music'")] 

In [41]:
data_two

,Unnamed: 0,genre,artist_name,track_name,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,duration_s,Unnamed: 0.1
0,1159,Alternative,Nickelback,How You Remind Me,75,0.00135,0.446,0.764,0.000000,0.0990,-5.042,0.0330,172.094,0.543,3730.666667,51.0
1,7012,Country,Andy Gibb,Shadow Dancing,46,0.14200,0.742,0.600,0.000000,0.0677,-9.647,0.0319,101.920,0.954,4600.000000,55.0
2,13501,Dance,Mariah Carey,We Belong Together,72,0.03580,0.838,0.469,0.000000,0.0928,-7.992,0.0835,139.975,0.778,3356.666667,15.0
3,13522,Dance,Mario,Let Me Love You,80,0.23500,0.656,0.578,0.000000,0.1180,-8.970,0.0922,94.514,0.556,4278.883333,65.0
4,13570,Dance,The Black Eyed Peas,I Gotta Feeling,70,0.08230,0.741,0.748,0.000000,0.3400,-6.299,0.0264,127.965,0.600,4818.883333,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176509,176509,Soul,H-Town,They Like It Slow,39,0.11300,0.611,0.493,0.000000,0.1050,-9.297,0.0354,115.920,0.443,4658.500000,NaN
176510,176510,Soul,John Legend,Quickly (feat. Brandy),39,0.23100,0.736,0.701,0.000000,0.2030,-4.345,0.1000,99.991,0.770,3711.116667,NaN
176511,176511,Soul,Belly,P.O.P.,43,0.10400,0.802,0.516,0.000485,0.1050,-9.014,0.2130,175.666,0.482,3352.883333,NaN
176512,176512,Soul,Jr Thomas & The Volcanos,Burning Fire,38,0.03290,0.785,0.683,0.000880,0.2370,-6.944,0.0337,113.830,0.969,4707.450000,NaN


In [42]:
data_two.genre.unique()

array(['Alternative', 'Country', 'Dance', 'Folk', 'Blues', 'Hip-Hop',
       'Pop', 'Rock', 'Soul', 'Movie', 'R&B', 'A Capella', 'Electronic',
       'Anime', 'Opera', 'Children’s Music', 'Rap', 'Indie', 'Classical',
       'Reggae', 'Reggaeton', 'Jazz', 'Ska', 'Soundtrack'], dtype=object)

In [43]:
data_two = data_two.drop(columns = ['Unnamed: 0','Unnamed: 0.1','track_name'])

In [44]:
data_two.dtypes

genre                object
artist_name          object
popularity            int64
acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
liveness            float64
loudness            float64
speechiness         float64
tempo               float64
valence             float64
duration_s          float64
dtype: object

In [ ]:
#data['match'] = data['match'].astype('bool')

In [45]:
ohe = OneHotEncoder()

data_object = data_two.select_dtypes('object')
ohe.fit(data_object)
codes = ohe.transform(data_object).toarray()
feature_names = ohe.get_feature_names(['genre','artist_name'])
data1 = pd.concat([data_two.select_dtypes(exclude='object'),pd.DataFrame(codes, columns = feature_names).astype(int)], axis=1)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [47]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167515 entries, 0 to 176513
Columns: 13769 entries, popularity to artist_name_정진우
dtypes: float64(13769)
memory usage: 17.2 GB


In [48]:
data1.columns

Index(['popularity', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence',
       ...
       'artist_name_彩音', 'artist_name_春音あいら(CV.阿澄佳奈)',
       'artist_name_棗坂シキ(CV.平川大輔)', 'artist_name_楚神ウリエ(CV.近藤隆)',
       'artist_name_立華リツカ(CV.茜屋日海夏)', 'artist_name_落日飛車 Sunset Rollercoaster',
       'artist_name_鉤貫レム(CV.斉藤壮馬)', 'artist_name_黒崎真音',
       'artist_name_알레프 ALEPH', 'artist_name_정진우'],
      dtype='object', length=13769)

In [49]:
data1 = data1.dropna()

In [50]:
# data1.to_csv('spotify_ohe_data.csv')

In [51]:
train, test = train_test_split(data1, test_size=0.3, random_state=1)

y_train = train['popularity'].to_numpy()
X_train = train.copy().drop(columns = ['popularity']).to_numpy()
y_test = test['popularity'].to_numpy()
X_test = test.copy().drop(columns = ['popularity']).to_numpy()

In [52]:
def algo_test(x_train,y_train,x_test,y_test):
        L=LinearRegression()
        R=Ridge()
        Lass=Lasso()
        E=ElasticNet()
        ETR=ExtraTreeRegressor()
        GBR=GradientBoostingRegressor()
        kn=KNeighborsRegressor()
        dt=DecisionTreeRegressor()
        xgb=XGBRegressor()
        
        algos=[L,R,Lass,E,ETR,GBR,kn,dt,xgb]
        algo_names=['Linear','Ridge','Lasso','ElasticNet','Extra Tree','Gradient Boosting','KNeighborsRegressor','Decision Tree','XGBRegressor']
        
        
        r_squared= []
        rmse= []
        mae= []
        
        result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)
        
        for algo in algos:
            p=algo.fit(x_train,y_train).predict(x_test)
            r_squared.append(r2_score(y_test,p))
            rmse.append(mean_squared_error(y_test,p)**.5)
            mae.append(mean_absolute_error(y_test,p))
        
        result.R_Squared=r_squared
        result.RMSE=rmse
        result.MAE=mae
        
        rtable=result.sort_values('R_Squared',ascending=False)
        return rtable

In [53]:
algo_test(X_train, y_train,X_test,y_test)

,R_Squared,RMSE,MAE
XGBRegressor,6.415794e-01,1.030182e+01,7.572570e+00
Gradient Boosting,5.547619e-01,1.148190e+01,8.683454e+00
Ridge,4.945655e-01,1.223348e+01,9.166605e+00
Extra Tree,4.125638e-01,1.318857e+01,8.996546e+00
Decision Tree,3.715123e-01,1.364162e+01,9.463079e+00
ElasticNet,1.216936e-01,1.612651e+01,1.309975e+01
Lasso,1.056013e-01,1.627358e+01,1.324749e+01
KNeighborsRegressor,4.765878e-03,1.716643e+01,1.373726e+01
Linear,-4.334839e+12,3.582646e+07,2.045340e+06
